# Analyse Isolated Star Sources

 work with Weakly_2024_07
- use jupyter kernel LSST


- author : Sylvie Dagoret-Campagne
- affiliation : IJCLab
- creation date : 2024/02/25
- update : 2024/02/25


In [1]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from mpl_toolkits.axes_grid1 import make_axes_locatable
from matplotlib.colors import LogNorm,SymLogNorm


import matplotlib.ticker                         # here's where the formatter is
from matplotlib.ticker import (MultipleLocator, FormatStrFormatter,
                               AutoMinorLocator)
from astropy.visualization import (MinMaxInterval, SqrtStretch,ZScaleInterval,PercentileInterval,
                                   ImageNormalize,imshow_norm)
from astropy.visualization.stretch import SinhStretch, LinearStretch,AsinhStretch,LogStretch
from astropy.io import fits
import astropy.coordinates as coord
import astropy.units as u
import pandas as pd

import matplotlib.ticker                         # here's where the formatter is
import os
import re
import pandas as pd

plt.rcParams["figure.figsize"] = (4,3)
plt.rcParams["axes.labelsize"] = 'xx-large'
plt.rcParams['axes.titlesize'] = 'xx-large'
plt.rcParams['xtick.labelsize']= 'xx-large'
plt.rcParams['ytick.labelsize']= 'xx-large'

In [2]:
#filesource_in="preSourceTable_202312.csv"
filesource_in="isolated_star_sources_202402.csv"

In [7]:
df = pd.read_csv(filesource_in,index_col=0)

In [8]:
df

,index,sourceId,visit,detector,ra,dec,physical_filter,band,apFlux_35_0_instFlux,apFlux_35_0_instFluxErr,x,y,apFlux_50_0_instFlux,apFlux_50_0_instFluxErr,apFlux_50_0_flag,localBackground_instFlux,localBackground_flag,source_row,obj_index,tract
0,22,177998123870715927,2023110800359,0,12.551369,-72.895434,empty~SDSSi_65mm,i,13212.871783,691.849487,3425.280389,74.046983,12824.632432,981.497251,False,0.614295,False,22,0,383
1,40,177998133534392361,2023110800368,0,12.551391,-72.895429,empty~SDSSi_65mm,i,13406.229133,636.976465,1768.346132,90.893416,12842.050168,902.685648,False,0.358499,False,40,0,383
2,161,178173923324592290,2023111300245,0,12.551398,-72.895433,empty~SDSSi_65mm,i,12472.655812,688.113400,3087.987781,363.749349,12089.859368,980.233677,False,0.181927,False,161,0,383
3,121,178173926545817722,2023111300248,0,12.551393,-72.895434,empty~SDSSi_65mm,i,13557.406984,685.153762,2535.062299,372.431613,14818.249205,975.847251,False,0.619669,False,121,0,383
4,21,178209144203902998,2023111400279,0,12.551398,-72.895448,empty~SDSSi_65mm,i,10620.340441,712.353010,2919.028476,159.923170,9525.332909,1011.543900,False,0.866852,False,21,0,383
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
152519,55,179546353280483384,2023122200468,0,122.636921,-36.166699,empty~SDSSy_65mm,y,6323.352977,561.712576,4029.246388,467.715657,NaN,NaN,True,-0.122081,False,55,1345,3864
152520,8,178913118334746633,2023120400546,0,122.627452,-36.261952,empty~SDSSy_65mm,y,6154.753825,537.923886,3854.589084,69.404199,6039.552589,767.419909,False,-0.111863,False,8,1346,3864
152521,534,179546353280483863,2023122200468,0,122.584535,-36.077190,empty~SDSSy_65mm,y,6814.557447,566.342704,2482.635397,3855.480970,6599.599039,804.355992,False,0.074439,False,534,1347,3864
152522,18,179300100256825363,2023121500503,0,122.571455,-36.266611,empty~SDSSy_65mm,y,7482.109294,647.611675,3342.339900,156.727808,9131.216602,923.076524,False,0.378480,False,18,1348,3864


In [ ]:
def convert_fluxtomag(x) :
    """
    The object and source catalogs store only fluxes. There are hundreds of flux-related columns, 
    and to store them also as magnitudes would be redundant, and a waste of space.
    All flux units are nanojanskys. The AB Magnitudes Wikipedia page provides a concise resource 
    for users unfamiliar with AB magnitudes and jansky fluxes. To convert to AB magnitudes use:
    As demonstrated in Section 2.3.2, to add columns of magnitudes after retrieving columns of flux, users can do this:
    results_table['r_calibMag'] = -2.50 * numpy.log10(results_table['r_calibFlux']) + 31.4
    results_table['r_cModelMag'] = -2.50 * numpy.log10(results_table['r_cModelFlux']) + 31.4
    (from DP0 tutorial)
    """
    return -2.50 * np.log10(x) + 31.4

In [ ]:
df_sel["psfMag"] =  df_sel['psfFlux'].map(convert_fluxtomag)
#df_sel["gaussianMag"] =  df_sel['gaussianFlux'].map(lambda x:-2.50 * np.log10(x) + 31.4)
df_sel["gaussianMag"] =  df_sel['gaussianFlux'].map(convert_fluxtomag)

In [ ]:
df_sel = df_sel[df_sel['extendedness_flag']==False & df_sel['detect_isPrimary']]

In [ ]:
df_sel.dropna(axis=0,inplace = True)

In [ ]:
df_sel

In [ ]:
df_sel.plot.scatter('x','y',c='ccdVisitId',colormap='viridis')

In [ ]:
df_sel.plot.scatter('ra','dec',c='ccdVisitId',colormap='hsv',grid=True,figsize=(10,5))

In [ ]:
df_sel.plot.scatter('coord_ra','coord_dec',c='ccdVisitId',colormap='hsv',grid=True,figsize=(10,5))

In [ ]:
df_sel['coord_ra'].plot(kind="hist",bins=50 ,range=(0,360),logy=True,title="coord_ra",grid=True,figsize=(10,5))

In [ ]:
df_sel['coord_dec'].plot(kind="hist",bins=50 ,range=(-90,10),logy=True,title="coord_dec",grid=True,figsize=(10,5))

In [ ]:
cut_skyloc = ((df_sel['dec'] < -30) & (df_sel['dec'] > -40) & (df_sel['ra'] > 100) & (df_sel['ra'] < 150))

In [ ]:
df_sel = df_sel[cut_skyloc]

In [ ]:
df_sel['psfMag'].plot(kind="hist",bins=50,range=(5,30),logy=True,grid=True,title="psMag")

In [ ]:
df_sel['gaussianMag'].plot(kind="hist",bins=50 ,range=(5,30),logy=True,grid=True,title="gaussianMag")

In [ ]:
df_sel

In [ ]:
#df_sel.groupby(df_sel['dateobs'].dt.date).size().plot()
datetime = pd.to_datetime(df_sel['dateobs'])

In [ ]:
df_sel['dateobs'].max()

In [ ]:
df_sel["date"] = df["dateobs"].astype("datetime64")

In [ ]:
df_sel

In [ ]:
df_time = df_sel[["date"]]

In [ ]:
df_time

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(1,1,1)
ax = (df_time["date"].groupby(df_time["date"].dt.minute)
                         .count()).plot(kind="bar")